In [ ]:
from google.colab import auth

# Aunthenticate to Google Colab
auth.authenticate_user()
project_id="<project id>"

#Configure Google Cloud
!gcloud config set project {project_id}

'''
  Enable Google SQL Admin API Services
'''
# Enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

Updated property [core/project].


In [ ]:
!pip install cloud-sql-python-connector #package to connect SQL-Python in Cloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.5/195.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.32.0 which is incompatible.


In [ ]:
!pip install pymysql #Python pkg that creates API access to SQL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
'''
  Drop DB if exists and create it afresh
'''
db_name='phonepe-bn'
instance_name = "<your mysql instance name>"
!gcloud sql databases delete {db_name} --instance={instance_name}

print(f'''Database '{db_name}' deleted successfully !!''')

The database will be deleted. Any data stored in the database will be destroyed.
 You cannot undo this action.

Do you want to continue (Y/n)?  Y

Deleted database [phonepe-bn].
Database 'phonepe-bn' deleted successfully !!


In [ ]:
!gcloud sql databases create {db_name} --instance={instance_name}
print(f'''Database '{db_name}' created successfully !!''')

In [ ]:
'''
  Connect to Cloud SQL DB to create the schema for 'phonepe-bn' database
'''
region = 'us-central1'
project_id="<your GCP project name"

from pp_dbconnect import PPDbConnector
cloudsql_conn_name = f'''{project_id}:{region}:{instance_name}'''
db_user= '<your sql login>'
db_password='<your password>'
pool = PPDbConnector(cloudsql_conn_name,db_user,db_password,db_name)

In [ ]:
'''
  Create the required tables for 'phonepe' database
'''
import sqlalchemy
with pool.connect() as db:
  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS geo (
        id INT NOT NULL,
        geo_type ENUM('CON','STA','DIS','CIT','PIN'),
        name VARCHAR(255) NOT NULL,
        latitude INT,
        longitude INT,
        parent_id INT,
        PRIMARY KEY (id),
        INDEX name (name));'''))

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS transaction_agg (
        id INT NOT NULL,
        year INT NOT NULL,
        quarter INT NOT NULL,
        geo_id INT NOT NULL,
        category VARCHAR(255) NOT NULL,
        stat_type VARCHAR(50) NOT NULL,
        count BIGINT NOT NULL DEFAULT 0,
        amount FLOAT NOT NULL DEFAULT 0,
        PRIMARY KEY (id),
        CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,category),
        INDEX rec_index (year,quarter,geo_id),
        FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS transaction_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS transaction_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          category VARCHAR(255) NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,category),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id, top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          reg_users INT NOT NULL DEFAULT 0,
          app_opens BIGINT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS device_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          brand VARCHAR(255) NOT NULL,
          count BIGINT NOT NULL,
          percentage INT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,brand),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          reg_users INT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          reg_users INT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

# Commit the above changes to DB
db.commit()
print( f'''Tables for '{db_name}' database successfully created !! ''')
db.close()

Successfully connected to 'phonepe-bn' database!! 
Tables for 'phonepe-bn' database successfully created !! 
